In [ ]:
import pandas as pd
import time
import os
import pyodbc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [ ]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("--verbose")
  options.add_argument("--no-sandbox")
  options.add_argument("--headless")
  options.add_argument("--disable-gpu")
  options.add_argument("--window-soze = 1920, 1200")
  options.add_argument("--disable-dev-shm-usage")
  driver = webdriver.Chrome(options = options)
  return driver

In [ ]:
driver = web_driver()

In [ ]:
url = "https://www.ccilindia.com/web/ccil/security-wise-repo-market-summary"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.1)

# create dataframe
df = pd.DataFrame()

# for each page in table
i = 1

#end page number
end_page = int(driver.find_elements(By.XPATH, '//*[@id="securityWiseRepoTable_paginate"]/span/a[6]')[0].text.splitlines()[0])

while (i <= end_page):

  #get list of rows of data
  rows = driver.find_elements(By.XPATH, '//*[@id="securityWiseRepoTable"]/tbody')[0].text.splitlines()

  for row in rows:
    #split first 2 elements in each row by space
    first_two = row.split(' ', 2)
    #split last 9 elements in each row by space
    last_nine = first_two[2].rsplit(' ', 9)
    #concat all elements into new row and add to df
    adjusted_row = [first_two[0], first_two[1], last_nine[0], last_nine[1], last_nine[2], last_nine[3], last_nine[4], last_nine[5],
                    last_nine[6], last_nine[7], last_nine[8], last_nine[9]]
    df = pd.concat([df, pd.DataFrame([adjusted_row])],axis=0, ignore_index=True)


  # get next button element
  next = driver.find_element(By.XPATH, '//*[@id="securityWiseRepoTable_next"]')
  # create action chain object
  action = ActionChains(driver)
  # click the item
  action.click(on_element = next)
  # perform the operation
  action.perform()
  # waiting for next page in table to load
  time.sleep(0.6)

  #advance page
  i += 1

df.columns= ['Date', 'ISIN', 'Security Deposition', 'Maturity Date', 'Trades', 'Volume (Cr)', 'Open Yield', 'High Yield', 'Low Yield', 'Last Yield', 'Wtd Avg Yield', 'Wtd Avg Price']
# Close the WebDriver
driver.quit()

In [ ]:
#export csv file
df.to_csv(r'/home/briansim74/Desktop/Security_Wise_Holdings.csv', index = False)

In [ ]:
#connect to Azure SQL server using pyodbc
server = 'sqldateserver.database.windows.net'
database = 'SQLTest'
username = 'briansim74'
password = 'Shalom70!'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#delete old data in SQL database

#delete data in table
cursor.execute('''TRUNCATE TABLE dbo.SECURITY_WISE_HOLDINGS''')

#execute command
cnxn.commit()

In [ ]:
#BCP utility bulk insert updated CSV file into SQL server
os.system('/opt/mssql-tools18/bin/bcp dbo.SECURITY_WISE_HOLDINGS in "/home/briansim74/Desktop/Security_Wise_Holdings.csv" -S sqldateserver.database.windows.net -d SQLTest -U briansim74 -P Shalom70! -c -F 2 -t "," -r "0x0a"')

In [ ]:
#close cursor
cursor.close()